In [4]:
from vllm import LLM, SamplingParams
from dotenv import load_dotenv

load_dotenv()

# llama-3-70 quantized
#llm = LLM('study-hjt/Meta-Llama-3-70B-Instruct-GPTQ-Int8', gpu_memory_utilization=0.9, tensor_parallel_size=8, enforce_eager=False, quantization="gptq")
#llm = LLM('meta-llama/Llama-3.2-1B-Instruct' enforce_eager=False, dtype="half")
llm = LLM("meta-llama/Llama-3.2-3B-Instruct", enforce_eager=False, dtype="half")




Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/rodri/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/.no_exist/0cb88a4f764b7a12671c53f0838cd831a0843b95/preprocessor_config.json'


WARNING 11-28 12:05:45 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 11-28 12:05:45 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
WARNING 11-28 12:05:45 arg_utils.py:1013] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-28 12:05:45 config.py:1136] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-28 12:05:45 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=None,

PermissionError: [Errno 13] Permission denied: '/home/rodri/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/refs/main'

In [ ]:
import random
random_seed = random.randint(0, 10000)
random.seed(random_seed)

print(f"seed: {random_seed}")

uvl_grammar_no_indent = r"""
?start: _NL* featuremodel

featuremodel: "features" _NL [feature+]
feature: NAME _NL (group+)?

group: "or" groupspec          -> or_group
    | "alternative" groupspec -> alternative_group
    | "optional" groupspec    -> optional_group
    | "mandatory" groupspec   -> mandatory_group
    | cardinality groupspec    -> cardinality_group

groupspec: _NL feature+

cardinality: "[" INT (".." (INT | "*"))? "]"

%import common.INT
%import common.CNAME -> NAME
%import common.WS_INLINE
%ignore WS_INLINE

_NL: /\r?\n[ \t]*/
"""

uvl_grammar = r"""
?start: _NL* featuremodel

featuremodel: "features" _NL ["->" feature+ "<-"]
feature: NAME _NL ("->" group+ "<-")?

group: "or" groupspec          -> or_group
    | "alternative" groupspec -> alternative_group
    | "optional" groupspec    -> optional_group
    | "mandatory" groupspec   -> mandatory_group
    | cardinality groupspec    -> cardinality_group

groupspec: _NL "->" feature+ "<-"

cardinality: "[" INT (".." (INT | "*"))? "]"

%import common.INT
%import common.CNAME -> NAME
%import common.WS_INLINE
%ignore WS_INLINE

_NL: /\r?\n[ \t]*/
"""

uvl_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for creating sintactically correct expressions similar to these examples:

features
    Sandwich
        mandatory
            Bread
        optional
            Sauce
                alternative
                    Ketchup
                    Mustard
            Cheese
constraints
    Ketchup => Cheese
,
features
	SmartWatch
		mandatory
			Functionalities 
				mandatory
					FitnessMonitor
					SleepTracker
					VibrateAlert
						mandatory
							Call
							Notification
			Sensors
				mandatory
					Pedometer
					Accelerometer
				optional
					HeartRateSensor
			Connectivity
				mandatory
					BT40


<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Write new, valid expressions for a computer: 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""


import time
start_time = time.perf_counter()
grammar = uvl_grammar
prompt = uvl_prompt
# llama-3 8B
#MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"
#llm = LLM(MODEL, gpu_memory_utilization=1, tensor_parallel_size=8, enforce_eager=False, dtype="half") 


sampling_params = SamplingParams(
        max_tokens=200,
        temperature=1,
        top_p=0.95,
        min_tokens=200,
    )


outputs = llm.generate(
    prompts=prompt,
    sampling_params=sampling_params,
    guided_options_request=dict(guided_grammar=grammar))

elapsed_time = time.perf_counter() - start_time
print(f'Elapsed time for inference: {elapsed_time} seconds')

from lark import Lark, exceptions
from lark.indenter import Indenter


def test(generation: str, parser):
    print(parser.parse(generation).pretty())

"""
class TreeIndenter(Indenter):
    NL_type = '_NL'
    OPEN_PAREN_types = []
    CLOSE_PAREN_types = []
    INDENT_type = '_INDENT'
    DEDENT_type = '_DEDENT'
    tab_len = 8
"""

#parser = Lark(grammar, parser='lalr', postlex=TreeIndenter())
parser = Lark(grammar, parser='lalr')

print("Grammar is well-written.")

for output in outputs:
    prompt = output.prompt

    generated_text = output.outputs[0].text
    print(f"Generated text without parser: {generated_text!r}")

    try:
        # Parse a generation
        test(generation=generated_text, parser=parser)

    except:
        print("Generation not grammatically valid")

seed: 9593
rog0d: First step -> Exclusive guided decodings
rog0d: Second step -> validate and add request
rog0d: third step-> Decoding backend
rog0d: fourth step-> outlines backend
rog0d: fifth step-> outlines operation
rog0d: sixth step-> check outlines guiding options
rog0d: seventh step-> CFGLogitsProcessor class: CFGuide to BaseLogitsProcessor
rog0d: ninth step-> _init_ CFGGuide


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

rog0d: eighth step-> _call_ BaseLogitsProcessor


Processed prompts: 100%|██████████| 1/1 [00:56<00:00, 56.50s/it, est. speed input: 3.68 toks/s, output: 3.54 toks/s]

Elapsed time for inference: 57.09211522806436 seconds
Grammar is well-written.
Generated text without parser: 'features\n->Feature1\n   ->mandatory\n       ->FeatureA\n       ->optional\n           ->FeatureB\n           ->optional\n               ->FeatureC\n<-<-<-<-<-<-<-               \t\t   \t\t\t        \t      \t            \t  \t        \t      \t\t    \t      \t        \t\t    \t        \t      \t        \t\t    \t  \t        \t      \t\t    \t      \t        \t\t    \t        \t      \t        \t\t    \t  \t        \t      \t\t    \t      \t        \t\t    \t        \t      \t        \t\t    \t  \t        \t      \t\t    \t      \t        \t\t    \t        \t      \t        \t\t    \t  \t        \t      \t\t    \t      \t        \t\t    \t        \t      \t        \t\t    \t\t\t        \t      \t\t    \t      \t        \t\t    \t        \t      \t        \t\t    \t  \t        \t      \t\t    \t      \t        \t\t    \t        \t      \t        \t\t    \t  \t        \t      \t

In [16]:
print(f"nº of generated tokens: {str(len(outputs[0].outputs[0].token_ids))}")
print(f"generated tokens id: {outputs[0].outputs[0].token_ids}")


nº of generated tokens: 200
generated tokens id: array('l', [13043, 82, 198, 405, 14180, 16, 198, 262, 405, 81216, 198, 286, 405, 14180, 32, 198, 286, 405, 13099, 198, 310, 405, 14180, 33, 198, 310, 405, 13099, 198, 394, 405, 14180, 34, 198, 46442, 46442, 46442, 46442, 27, 12, 46442, 27, 12, 2342, 220, 6585, 573, 17815, 52470, 36657, 19827, 17815, 87459, 7938, 52470, 38020, 7938, 17815, 996, 24311, 57646, 19827, 17815, 87459, 7938, 52470, 38020, 7938, 17815, 996, 24311, 57646, 19827, 17815, 87459, 7938, 52470, 38020, 7938, 17815, 996, 24311, 57646, 19827, 17815, 87459, 7938, 52470, 38020, 7938, 17815, 996, 24311, 57646, 19827, 17815, 87459, 7938, 52470, 38020, 7938, 17815, 996, 24311, 18952, 573, 17815, 87459, 7938, 52470, 38020, 7938, 17815, 996, 24311, 57646, 19827, 17815, 87459, 7938, 52470, 38020, 7938, 17815, 996, 24311, 57646, 19827, 17815, 87459, 7938, 52470, 38020, 7938, 17815, 996, 24311, 99978, 15646, 7938, 52470, 38020, 7938, 17815, 996, 24311, 57646, 19827, 17815, 87459, 79

In [24]:
gen = outputs[0].outputs[0].text
print(f"{str(gen)}")

print(parser.parse(gen).pretty())



features
->Feature1
   ->mandatory
       ->FeatureA
       ->optional
           ->FeatureB
           ->optional
               ->FeatureC
<-<-<-<-<-<-<-               		   			        	      	            	  	        	      		    	      	        		    	        	      	        		    	  	        	      		    	      	        		    	        	      	        		    	  	        	      		    	      	        		    	        	      	        		    	  	        	      		    	      	        		    	        	      	        		    	  	        	      		    	      	        		    	        	      	        		    			        	      		    	      	        		    	        	      	        		    	  	        	      		    	      	        		    	        	      	        		    	  	        	      		    	      	        		    	        	      	        		    		    		    	      	        		    	        	      	        		    	  	        	      		    	      	        		    	        	      	        		    	  	        	      		    	  